<a href="https://colab.research.google.com/github/Abdul-Wahab133/first-project-of-langchain/blob/main/Project_02_RAG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -Uq langchain-pinecone langchain-google-genai

from pinecone import Pinecone, ServerlessSpec

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 41.5/41.5 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 31.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 427.3/427.3 kB 18.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.5/87.5 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.5/49.5 kB 1.7 MB/s eta 0:00:00


In [ ]:
from google.colab import userdata
import os
os.environ['PINECONE_API_KEY'] = userdata.get('PINECONE_API_KEY')

pinecone_api_key = userdata.get('PINECONE_API_KEY')

pc = Pinecone(api_key=pinecone_api_key)

In [ ]:
import time
index_name = "project-2-rag"

existing_indexes = [index_info["project-2-rag"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)



In [ ]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions, known for their loyalty and friendliness.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Cats are independent pets that often enjoy their own space.",
        metadata={"source": "mammal-pets-doc"},
    ),
    Document(
        page_content="Goldfish are popular pets for beginners, requiring relatively simple care.",
        metadata={"source": "fish-pets-doc"},
    ),
    Document(
        page_content="Parrots are intelligent birds capable of mimicking human speech.",
        metadata={"source": "bird-pets-doc"},
    ),
    Document(
        page_content="Rabbits are social animals that need plenty of space to hop around.",
        metadata={"source": "mammal-pets-doc"},
    ),
]



In [ ]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import getpass
import os

os.environ['GOOGLE_API_KEY'] = userdata.get('GOOGLE_API_KEY')


embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004",google_api_key=userdata.get('GOOGLE_API_KEY'))


In [ ]:
from langchain_pinecone import PineconeVectorStore

vector_store = PineconeVectorStore.from_documents(
    documents,
    embedding = embeddings,
    index_name=index_name,
)

In [ ]:
from langchain_pinecone import PineconeVectorStore
vector_store = PineconeVectorStore(index=index, embedding = embeddings)



In [ ]:
vector_store.similarity_search_with_score("tell me about wahab")

[(Document(id='c09e1de7-ea02-4ec1-8f83-6567ae6eecab', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions, known for their loyalty and friendliness.'),
  0.404622316),
 (Document(id='f7b4921c-f7f0-4a6f-83c9-238b856d2ddd', metadata={'source': 'mammal-pets-doc'}, page_content='Rabbits are social animals that need plenty of space to hop around.'),
  0.386931747),
 (Document(id='b2f6c6f6-7c59-4c15-b0b5-b0365a10d173', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners, requiring relatively simple care.'),
  0.377906144),
 (Document(id='ad583218-7017-4ed4-8fdb-f7884b08cdd5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.'),
  0.364877701)]

In [ ]:
embedding = embeddings.embed_query("cat")
embedding
#results = vector_store.similarity_search_by_vector(query_embedding)

[-0.014495342038571835,
 -0.004356286954134703,
 0.030960695818066597,
 -0.01937987469136715,
 0.046561747789382935,
 0.07971760630607605,
 0.044616784900426865,
 -0.019115572795271873,
 0.08993305265903473,
 0.05842595174908638,
 -0.0453537255525589,
 0.018237140029668808,
 0.08033973723649979,
 0.02752574160695076,
 -0.028318511322140694,
 -0.02716362290084362,
 0.006191493012011051,
 0.005197346676141024,
 -0.01692069135606289,
 0.03503800556063652,
 0.08894353359937668,
 -0.010495638474822044,
 0.04114319011569023,
 -0.01638908125460148,
 -0.03600003570318222,
 0.01174926944077015,
 0.03139335289597511,
 0.03141077607870102,
 -0.0031625358387827873,
 -0.016714120283722878,
 0.042587678879499435,
 0.04249170795083046,
 0.01343710720539093,
 0.02279532328248024,
 0.0567803755402565,
 -0.01019603107124567,
 -0.014729867689311504,
 0.06853161752223969,
 -0.008628561161458492,
 -0.022934652864933014,
 -0.03452526405453682,
 0.01392278727144003,
 -0.022488689050078392,
 0.021314928308129

In [ ]:
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vector_store.similarity_search).bind(k=1)

retriever.batch(["cats"])

[[Document(id='ad583218-7017-4ed4-8fdb-f7884b08cdd5', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets that often enjoy their own space.')]]

In [ ]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(model= "gemini-2.0-flash-exp",api_key=userdata.get('GOOGLE_API_KEY'))


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer the question using the provided context only.
{Question},
context :
{Context}
"""

In [74]:
prompt = ChatPromptTemplate.from_messages([("human",message)])

In [75]:
rag_chain = {"Context": retriever, "Question": RunnablePassthrough()} | prompt | llm

In [79]:
response = rag_chain.invoke("tell me about cat")

In [80]:
print(response.content)

Cats are independent pets that often enjoy their own space.
